In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from extract.locm2 import AP, Hypothesis, LOCM2
from traces import *
from observation import ActionObservation
from extract.model import *

In [3]:
def get_example_obs_bw_2traces(print_trace=False):
    objects = {
        "A": PlanningObject("unknown", "A"),
        "B": PlanningObject("unknown", "B"),
        "C": PlanningObject("unknown", "C"),
    }

    actions = {
        "unstackAB": Action("unstack", [objects["A"], objects["B"]]),
        "unstackBC": Action("unstack", [objects["B"], objects["C"]]),
        "unstackCB": Action("unstack", [objects["C"], objects["B"]]),
        "unstackBA": Action("unstack", [objects["B"], objects["A"]]),
        "unstackAC": Action("unstack", [objects["A"], objects["C"]]),
        "unstackCA": Action("unstack", [objects["C"], objects["A"]]),
        "stackAB": Action("stack", [objects["A"], objects["B"]]),
        "stackBA": Action("stack", [objects["B"], objects["A"]]),
        "stackAC": Action("stack", [objects["A"], objects["C"]]),
        "stackCA": Action("stack", [objects["C"], objects["A"]]),
        "stackBC": Action("stack", [objects["B"], objects["C"]]),
        "stackCB": Action("stack", [objects["C"], objects["B"]]),
        "putdownA": Action("putdown", [objects["A"]]),
        "putdownB": Action("putdown", [objects["B"]]),
        "putdownC": Action("putdown", [objects["C"]]),
        "pickupA": Action("pickup", [objects["A"]]),
        "pickupB": Action("pickup", [objects["B"]]),
        "pickupC": Action("pickup", [objects["C"]]),
    }

    input_action_seqs = '''
        unstack(A, B), putdown(A), pickup(B), stack(B,C), unstack(B,C), stack(B,C),pickup(A), stack(A,B)
        unstack(C, B), putdown(C), unstack(B,A), putdown(B), pickup(B), stack(B,C), pickup(A), stack(A,B)
        unstack(C,B), stack(C,A), pickup(B), putdown(B), pickup(B), stack(B,C)
        '''
       
    traces = TraceList(
        [
            Trace(
                [
                    Step(State({}), actions["unstackAB"], 1),
                    Step(State({}), actions["putdownA"], 2),
                    Step(State({}), actions["pickupB"], 3),
                    Step(State({}), actions["stackBC"], 4),
                    Step(State({}), actions["unstackBC"], 5),
                    Step(State({}), actions["stackBC"], 6),
                    Step(State({}), actions["pickupA"], 7),
                    Step(State({}), actions["stackAB"], 8),
                ]
            ),
            Trace(
                [
                    Step(State({}),actions['unstackCB'], 1),
                    Step(State({}),actions['putdownC'], 2),
                    Step(State({}),actions['unstackBA'], 3),
                    Step(State({}),actions['putdownB'], 4),
                    Step(State({}),actions['pickupB'], 5),
                    Step(State({}),actions['stackBC'], 6),
                    Step(State({}),actions['pickupA'], 7),
                    Step(State({}),actions['stackAB'], 8),

                ]
            ),
            Trace(
                [
                    Step(State({}), actions["unstackCB"], 1),
                    Step(State({}), actions["stackCA"], 2),
                    Step(State({}), actions["pickupB"], 3),
                    Step(State({}), actions["putdownB"], 4),
                    Step(State({}), actions["pickupB"], 5),
                    Step(State({}), actions["stackBC"], 6),
                ]
            ),
            
        ]
    )
    

    if print_trace:
        # traces.print()
        traces.print("detail")

    obs = traces.tokenize(ActionObservation)
    return obs

In [4]:
def get_example_obs(print_trace=False, ex=1):
    objects = {
        "c1": PlanningObject("container", "c1"),
        "c2": PlanningObject("container", "c2"),
        "c3": PlanningObject("container", "c3"),
        "j1": PlanningObject("jack", "j1"),
        "j2": PlanningObject("jack", "j2"),
        "wr1": PlanningObject("wrench", "wr1"),
        "wr2": PlanningObject("wrench", "wr2"),
    }
    fluents = {
        "open1": Fluent("open", [objects["c1"]]),
        "open2": Fluent("open", [objects["c2"]]),
        "open3": Fluent("open", [objects["c3"]]),
        "j1in": Fluent("in", [objects["j1"], objects["c1"]]),
        "j2in": Fluent("in", [objects["j2"], objects["c2"]]),
        "wr1in": Fluent("in", [objects["wr1"], objects["c1"]]),
        "wr2in": Fluent("in", [objects["wr2"], objects["c2"]]),
    }
    actions = {
        "open1": Action("open", [objects["c1"]]),
        "open2": Action("open", [objects["c2"]]),
        "open3": Action("open", [objects["c3"]]),
        "close1": Action("close", [objects["c1"]]),
        "close2": Action("close", [objects["c2"]]),
        "close3": Action("close", [objects["c3"]]),
        "fetchj1": Action("fetch_jack", [objects["j1"], objects["c1"]]),
        "fetchj2": Action("fetch_jack", [objects["j2"], objects["c2"]]),
        "putj1": Action("putaway_jack", [objects["j1"], objects["c1"]]),
        "putj2": Action("putaway_jack", [objects["j2"], objects["c2"]]),
        "fetchwr1": Action("fetch_wrench", [objects["wr1"], objects["c1"]]),
        "fetchwr2": Action("fetch_wrench", [objects["wr2"], objects["c2"]]),
        "putwr1": Action("putaway_wrench", [objects["wr1"], objects["c1"]]),
        "putwr2": Action("putaway_wrench", [objects["wr2"], objects["c2"]]),
        "closewr": Action("close", [objects["wr1"]]),
    }

    if ex == 1:
        # open(c1); fetch jack(j1,c1); fetch wrench(wr1,c1); close(c1);
        # open(c2); fetch wrench(wr2,c2); fetch jack(j2,c2); close(c2);
        # open(c3); close(c3)
        states_true = [
            ["open3", "j1in", "j2in", "wr1in", "wr2in"],
            ["open3", "open1", "j1in", "j2in", "wr1in", "wr2in"],
            ["open3", "open1", "j2in", "wr1in", "wr2in"],
            ["open3", "open1", "j2in", "wr2in"],
            ["open3", "j2in", "wr2in"],
            ["open3", "open2", "j2in", "wr2in"],
            ["open3", "open2", "j2in"],
            ["open3", "open2"],
            ["open3"],
            [],
            ["open3"],
        ]
        states = [
            State({fluent: name in state_true for name, fluent in fluents.items()})
            for state_true in states_true
        ]
        traces = TraceList(
            [
                Trace(
                    [
                        Step(states[0], actions["open1"], 1),
                        Step(states[1], actions["fetchj1"], 2),
                        Step(states[2], actions["fetchwr1"], 3),
                        Step(states[3], actions["close1"], 4),
                        Step(states[4], actions["open2"], 5),
                        Step(states[5], actions["fetchwr2"], 6),
                        Step(states[6], actions["fetchj2"], 7),
                        Step(states[7], actions["close2"], 8),
                        Step(states[9], actions["close3"], 9),
                        Step(states[8], actions["open3"], 10),
                        Step(states[10], None, 11),
                        # Step(states[10], actions["closewr"], 11),
                        # Step(states[11], None, 12),
                    ]
                ),
            ]
        )
    else:
        # open(c1); putaway jack(j1,c1); close(c1); open(c2); putaway jack(j2,c2);
        # open(c1); fetch jack(j1,c1); fetch wrench(wr1,c1);
        # fetch jack(j2,c2); close(c1);
        states_true = [
            ["wr1in", "wr2in"],
            ["open1", "wr1in", "wr2in"],
            ["open1", "wr1in", "wr2in", "j1in"],
            ["wr1in", "wr2in", "j1in"],
            ["open2", "wr1in", "wr2in", "j1in"],
            ["open2", "wr1in", "wr2in", "j1in", "j2in"],
            ["open1", "open2", "wr1in", "wr2in", "j1in", "j2in"],
            ["open1", "open2", "wr1in", "wr2in", "j2in"],
            ["open1", "open2", "wr2in", "j2in"],
            ["open1", "open2", "wr2in"],
            ["open2", "wr2in"],
        ]
        states = [
            State({fluent: name in state_true for name, fluent in fluents.items()})
            for state_true in states_true
        ]
        traces = TraceList(
            [
                Trace(
                    [
                        Step(states[0], actions["open1"], 1),
                        Step(states[1], actions["putj1"], 2),
                        Step(states[2], actions["close1"], 3),
                        Step(states[3], actions["open2"], 4),
                        Step(states[4], actions["putj2"], 5),
                        Step(states[5], actions["open1"], 6),
                        Step(states[6], actions["fetchj1"], 7),
                        Step(states[7], actions["fetchwr1"], 8),
                        Step(states[8], actions["fetchj2"], 9),
                        Step(states[9], actions["close1"], 10),
                    ]
                ),
            ]
        )

    if print_trace:
        # traces.print()
        traces.print("color")

    obs = traces.tokenize(ActionObservation)
    return obs

In [5]:
#obs_tracelist = get_example_obs_bw_2traces(True)
obs_tracelist = get_example_obs()


/home/uto/anu/codes/xLOCM/src/traces/trace_list.py:150: UserWarning: Invalid view detail. Defaulting to "details".
  warn(f'Invalid view {view}. Defaulting to "details".')


                                                       Trace                                                       
Attributes:                                                                                                        
  8 steps                                                                                                          
  0 fluents                                                                                                        
                                                       Steps                                                       
                     Step       State        Action                                                                
                        1                    unstack unknown A unknown B                                           
                        2                    putdown unknown A                                                     
                        3                    pickup unknown B                                                      
                        4                    stack unknown B unknown C                                             
                        5                    unstack unknown B unknown C                                           
                        6                    stack unknown B unknown C                                             
                        7                    pickup unknown A                                                      
                        8                    stack unknown A unknown B

                                                       Trace                                                       
Attributes:                                                                                                        
  8 steps                                                                                                          
  0 fluents                                                                                                        
                                                       Steps                                                       
                     Step       State        Action                                                                
                        1                    unstack unknown C unknown B                                           
                        2                    putdown unknown C                                                     
                        3                    unstack unknown B unknown A                                           
                        4                    putdown unknown B                                                     
                        5                    pickup unknown B                                                      
                        6                    stack unknown B unknown C                                             
                        7                    pickup unknown A                                                      
                        8                    stack unknown A unknown B

                                                       Trace                                                       
Attributes:                                                                                                        
  6 steps                                                                                                          
  0 fluents                                                                                                        
                                                       Steps                                                       
                     Step       State        Action                                                                
                        1                    unstack unknown C unknown B                                           
                        2                    stack unknown C unknown A                                             
                        3                    pickup unknown B                                                      
                        4                    putdown unknown B                                                     
                        5                    pickup unknown B                                                      
                        6                    stack unknown B unknown C

In [6]:
def test_get_sort(obs_tracelist, debug=False):
   

    sorts = LOCM2._get_sorts(obs_tracelist, debug)

    return sorts

In [7]:
sorts = test_get_sort(obs_tracelist, False)
print(sorts)

{'A': 1, 'C': 1, 'B': 1}


In [8]:
AML, obj_traces_overall = LOCM2._locm2_step1(obs_tracelist, sorts, True)

Sort.0 AML:


,unstack.0,putdown.0,pickup.0,stack.0
unstack.0,0,3,0,2
putdown.0,1,0,3,0
pickup.0,0,1,0,5
stack.0,1,0,3,0


Sort.1 AML:


,unstack.1,unstack.2,putdown.1,pickup.1,stack.1,stack.2
unstack.1,0,0,3,0,2,0
unstack.2,1,0,0,3,0,1
putdown.1,0,0,0,3,0,1
pickup.1,0,0,1,0,5,0
stack.1,1,0,0,0,0,3
stack.2,0,1,0,0,0,0


In [9]:
AML_with_holes = LOCM2._locm2_step2(AML, True)

Sort.1 AML with holes:


/home/uto/anu/codes/xLOCM/src/extract/locm2.py:503: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'hole' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df1.iloc[idx2,col] = 'hole'
/home/uto/anu/codes/xLOCM/src/extract/locm2.py:503: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'hole' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df1.iloc[idx2,col] = 'hole'


,unstack.1,unstack.2,putdown.1,pickup.1,stack.1,stack.2
unstack.1,0,0,3,0,2,0
unstack.2,1,0,0,3,0,1
putdown.1,hole,0,0,3,0,1
pickup.1,0,0,1,0,5,0
stack.1,1,0,0,hole,0,3
stack.2,0,1,0,0,0,0


In [10]:
H_per_sort = LOCM2._locm2_step3(AML_with_holes, True)

#holes in Sort.1: 2


In [11]:
transitions_per_sort = LOCM2._locm2_step4(AML_with_holes)
consecutive_transitions_per_sort = LOCM2._locm2_step5(AML_with_holes)

In [12]:
S = LOCM2._locm2_step6(AML, H_per_sort, transitions_per_sort, consecutive_transitions_per_sort)

### Sort.1

no holes
[]

Removed redundancy transition set list
[]


#### Final transition set list

[{stack.0, pickup.0, putdown.0, unstack.0}]


### Sort.2

...

Checking candidate set *{stack.2, pickup.1, stack.1}* of **Sort.2** for well formedness and Validity

,stack.2,pickup.1,stack.1
stack.2,0,0,0
pickup.1,0,0,5
stack.1,3,0,0


0 1
0 2
1 2
This subset is well-formed.
This subset is valid.
Adding this subset {stack.2, pickup.1, stack.1} to the locm2 transition set.
Hole that is covered now:
[pickup.1, stack.1]
[{putdown.1, unstack.1, pickup.1}, {stack.2, pickup.1, stack.1}]

Removed redundancy transition set list
[{putdown.1, unstack.1, pickup.1}, {stack.2, pickup.1, stack.1}]


#### Final transition set list

[{putdown.1, unstack.1, pickup.1}, {stack.2, pickup.1, stack.1}, {pickup.1, unstack.2, stack.2, stack.1, putdown.1, unstack.1}]


In [13]:
TS_overall, ap_state_pointers, OS = LOCM2._step1(obj_traces_overall, sorts, S, AML, True)

TS_overall: 
 [{FSM.0 @ Sort.0: {zero zero: [unstack.0, putdown.0, pickup.0, stack.0, unstack.0, stack.0, pickup.0, stack.0]}, FSM.0 @ Sort.1: {unknown A: [unstack.1, putdown.1, pickup.1], unknown B: [pickup.1, unstack.1], unknown C: []}, FSM.1 @ Sort.1: {unknown A: [pickup.1, stack.1], unknown B: [pickup.1, stack.1, stack.1, stack.2], unknown C: [stack.2, stack.2]}, FSM.2 @ Sort.1: {unknown A: [unstack.1, putdown.1, pickup.1, stack.1], unknown B: [unstack.2, pickup.1, stack.1, unstack.1, stack.1, stack.2], unknown C: [stack.2, unstack.2, stack.2]}}, {FSM.0 @ Sort.0: {zero zero: [unstack.0, putdown.0, unstack.0, putdown.0, pickup.0, stack.0, pickup.0, stack.0]}, FSM.0 @ Sort.1: {unknown C: [unstack.1, putdown.1], unknown B: [unstack.1, putdown.1, pickup.1], unknown A: [pickup.1]}, FSM.1 @ Sort.1: {unknown C: [stack.2], unknown B: [pickup.1, stack.1, stack.2], unknown A: [pickup.1, stack.1]}, FSM.2 @ Sort.1: {unknown C: [unstack.1, putdown.1, stack.2], unknown B: [unstack.2, unstack.1, 

In [14]:
HS = LOCM2._step3(TS_overall, ap_state_pointers, OS, sorts, AML, True)

Adding Hypo:
<
  S=0
  B={'action': stack unknown B unknown C, 'pos': 1, 'sort': 1}
  k=1
  k_=2
  C={'action': stack unknown A unknown B, 'pos': 2, 'sort': 1}
  l=2
  l_=1
  G=1
  G_=1
  fsm={'sort': 1, 'index': 1}
>
Adding Hypo:
<
  S=2
  B={'action': stack unknown B unknown C, 'pos': 1, 'sort': 1}
  k=1
  k_=2
  C={'action': unstack unknown B unknown C, 'pos': 1, 'sort': 1}
  l=1
  l_=2
  G=1
  G_=1
  fsm={'sort': 1, 'index': 2}
>
Adding Hypo:
<
  S=1
  B={'action': unstack unknown B unknown C, 'pos': 1, 'sort': 1}
  k=1
  k_=2
  C={'action': stack unknown B unknown C, 'pos': 1, 'sort': 1}
  l=1
  l_=2
  G=1
  G_=1
  fsm={'sort': 1, 'index': 2}
>
Adding Hypo:
<
  S=2
  B={'action': stack unknown B unknown C, 'pos': 1, 'sort': 1}
  k=1
  k_=2
  C={'action': stack unknown A unknown B, 'pos': 2, 'sort': 1}
  l=2
  l_=1
  G=1
  G_=1
  fsm={'sort': 1, 'index': 2}
>
Adding Hypo:
<
  S=0
  B={'action': stack unknown B unknown C, 'pos': 2, 'sort': 1}
  k=2
  k_=1
  C={'action': unstack unkn

In [15]:
Bindings = LOCM2._step4(HS, debug=True)

defaultdict(<class 'dict'>,
            {FSM.2 @ Sort.1: {0: [Binding(hypothesis=<
  S=0
  B={'action': stack unknown B unknown C, 'pos': 2, 'sort': 1}
  k=2
  k_=1
  C={'action': unstack unknown B unknown C, 'pos': 2, 'sort': 1}
  l=2
  l_=1
  G=1
  G_=1
  fsm={'sort': 1, 'index': 2}
>, param=0)],
                              2: [Binding(hypothesis=<
  S=2
  B={'action': unstack unknown B unknown C, 'pos': 2, 'sort': 1}
  k=2
  k_=1
  C={'action': stack unknown B unknown C, 'pos': 2, 'sort': 1}
  l=2
  l_=1
  G=1
  G_=1
  fsm={'sort': 1, 'index': 2}
>, param=0),
                                  Binding(hypothesis=<
  S=2
  B={'action': stack unknown B unknown C, 'pos': 1, 'sort': 1}
  k=1
  k_=2
  C={'action': unstack unknown B unknown C, 'pos': 1, 'sort': 1}
  l=1
  l_=2
  G=1
  G_=1
  fsm={'sort': 1, 'index': 2}
>, param=1)]}})


In [16]:
Bindings = LOCM2._step5(HS, Bindings, True)

In [17]:
fluents, actions = LOCM2._step7(OS, ap_state_pointers, sorts,Bindings,{}, debug=True)

bindings:
{FSM.2 @ Sort.1: {0: [Binding(hypothesis=<
  S=0
  B={'action': stack unknown B unknown C, 'pos': 2, 'sort': 1}
  k=2
  k_=1
  C={'action': unstack unknown B unknown C, 'pos': 2, 'sort': 1}
  l=2
  l_=1
  G=1
  G_=1
  fsm={'sort': 1, 'index': 2}
>, param=0)]}}

fluents:
{(fsmFSM.0 @ Sort.1_state2 sort1),
 (fsmFSM.0 @ Sort.1_state1 sort1),
 (fsmFSM.1 @ Sort.1_state2 sort1),
 (fsmFSM.1 @ Sort.1_state0 sort1),
 (fsmFSM.0 @ Sort.0_state1 sort0),
 (fsmFSM.0 @ Sort.0_state0 sort0),
 (fsmFSM.0 @ Sort.1_state0 sort1),
 (fsmFSM.1 @ Sort.1_state1 sort1),
 (fsmFSM.2 @ Sort.1_state0 sort1 sort1),
 (fsmFSM.1 @ Sort.1_state3 sort1),
 (fsmFSM.2 @ Sort.1_state2 sort1),
 (fsmFSM.2 @ Sort.1_state1 sort1)}

actions:
{(unstack sort0 sort1 sort1),
 (putdown sort0 sort1),
 (stack sort0 sort1 sort1),
 (pickup sort0 sort1)}



In [18]:
state_machines = LOCM2.get_state_machines(ap_state_pointers, OS, Bindings)
for sm in state_machines:
    sm.render(view=True)

In [19]:
model = Model(fluents, actions)

In [20]:
model.to_pddl('locm')

putdown
{(fsmFSM.2 @ Sort.1_state1 sort1), (fsmFSM.0 @ Sort.1_state0 sort1), (fsmFSM.0 @ Sort.0_state0 sort0)}
[x0 (sort0), x1 (sort1)]
(fsmFSM.2 @ Sort.1_state1 sort1)
[1]
(fsmFSM.0 @ Sort.1_state0 sort1)
[1]
(fsmFSM.0 @ Sort.0_state0 sort0)
[0]
stack
{(fsmFSM.2 @ Sort.1_state2 sort1), (fsmFSM.1 @ Sort.1_state3 sort1), (fsmFSM.2 @ Sort.1_state1 sort1), (fsmFSM.0 @ Sort.0_state0 sort0), (fsmFSM.1 @ Sort.1_state0 sort1)}
[x0 (sort0), x1 (sort1), x2 (sort1)]
(fsmFSM.2 @ Sort.1_state2 sort1)
[2]
(fsmFSM.1 @ Sort.1_state3 sort1)
[1]
(fsmFSM.2 @ Sort.1_state1 sort1)
[1]
(fsmFSM.0 @ Sort.0_state0 sort0)
[0]
(fsmFSM.1 @ Sort.1_state0 sort1)
[2]
pickup
{(fsmFSM.0 @ Sort.1_state2 sort1), (fsmFSM.0 @ Sort.0_state1 sort0), (fsmFSM.2 @ Sort.1_state2 sort1), (fsmFSM.1 @ Sort.1_state2 sort1)}
[x0 (sort0), x1 (sort1)]
(fsmFSM.0 @ Sort.1_state2 sort1)
[1]
(fsmFSM.0 @ Sort.0_state1 sort0)
[0]
(fsmFSM.2 @ Sort.1_state2 sort1)
[1]
(fsmFSM.1 @ Sort.1_state2 sort1)
[1]
unstack
{(fsmFSM.0 @ Sort.1_state1 so